In [66]:
import math, json, os, sys

import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing import image
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# making dataset using image data generator class keras

In [30]:
gen = keras.preprocessing.image.ImageDataGenerator()

In [31]:
TRAIN_DIR='data'


batches = gen.flow_from_directory(TRAIN_DIR, target_size=(224,224),
                                  
                                   class_mode='categorical', shuffle=True, batch_size=32)

Found 1734 images belonging to 3 classes.


# loading resnet

In [32]:
 model = keras.applications.resnet50.ResNet50()

In [33]:
 classes = list(iter(batches.class_indices))

In [34]:
classes

['covid', 'normal', 'pneumonia']

# Performing network surgery of Resnet 50 (fine tuning)

# removing output layer of resnet 50

In [35]:
model.layers

In [36]:
model.layers.pop()

# freezing all layers of resnet with weights

In [37]:
for layer in model.layers:
    layer.trainable =False
        

# attaching out dense layer to resnet frozen net

In [38]:
# taking last layer of resnet as output of our inputs

In [47]:
last = model.layers[-1].output


In [40]:
# defining ourdense layer

In [48]:
x = Dense(len(classes), activation="softmax")(last)

In [49]:
final_model = Model([model.input],[x])
for c in batches.class_indices:    
    classes[batches.class_indices[c]] = c
    final_model.classes = classes

In [63]:
# compiling the model
final_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [64]:
final_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

# fitting data

In [56]:
final_model.fit(batches,steps_per_epoch=30, epochs=25)

Epoch 1/25
30/30 [==============================] - 27s 915ms/step - loss: 0.4573 - accuracy: 0.7927
Epoch 2/25
30/30 [==============================] - 33s 1s/step - loss: 0.4876 - accuracy: 0.7869
Epoch 3/25
30/30 [==============================] - 36s 1s/step - loss: 0.4531 - accuracy: 0.8042
Epoch 4/25
30/30 [==============================] - 35s 1s/step - loss: 0.4905 - accuracy: 0.7709
Epoch 5/25
30/30 [==============================] - 33s 1s/step - loss: 0.4982 - accuracy: 0.7827
Epoch 6/25
30/30 [==============================] - 36s 1s/step - loss: 0.4521 - accuracy: 0.7990
Epoch 7/25
30/30 [==============================] - 35s 1s/step - loss: 0.4787 - accuracy: 0.7667
Epoch 8/25
30/30 [==============================] - 34s 1s/step - loss: 0.4463 - accuracy: 0.8084
Epoch 9/25
30/30 [==============================] - 36s 1s/step - loss: 0.4421 - accuracy: 0.8212
Epoch 10/25
30/30 [==============================] - 37s 1s/step - loss: 0.4321 - accuracy: 0.8083
Epoch 11/25
30/3

In [73]:
def predict(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    preds = model.predict(x)
    return preds

In [75]:
predict(r'data\covid\1.CXRCTThoraximagesofCOVID-19fromSingapore.pdf-000-fig1b.png',final_model)

array([[4.2224515e-04, 9.8729986e-01, 1.2277914e-02]], dtype=float32)

In [57]:
val_batche = gen.flow_from_directory(TRAIN_DIR, target_size=(224,224),
                                  
                                   class_mode='categorical', shuffle=True, batch_size=32)

Found 1734 images belonging to 3 classes.


In [68]:
data_list = []
batch_index = 0

while batch_index <= val_batche.batch_index:
    data = val_batche.next()
    data_list.append(data[0])
    batch_index = batch_index + 1

# now, data_array is the numeric data of whole images
data_array = np.asarray(data_list)

In [72]:
val_batche.shape

AttributeError: 'DirectoryIterator' object has no attribute 'shape'

In [59]:

es = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    mode = 'min',
    verbose=1,
    patience = 40,
    restore_best_weights=True
)

In [77]:
final_model.fit(batches,steps_per_epoch=30, epochs=40,validation_data=val_batche,callbacks=[es],shuffle=True)

Epoch 1/40
30/30 [==============================] - 64s 2s/step - loss: 0.3387 - accuracy: 0.8630 - val_loss: 3.3741 - val_accuracy: 0.3737
Epoch 2/40
30/30 [==============================] - 118s 4s/step - loss: 0.3359 - accuracy: 0.8833 - val_loss: 4.3138 - val_accuracy: 0.3968
Epoch 3/40
30/30 [==============================] - 150s 5s/step - loss: 0.3343 - accuracy: 0.8640 - val_loss: 3.5939 - val_accuracy: 0.4181
Epoch 4/40
30/30 [==============================] - 171s 6s/step - loss: 0.3235 - accuracy: 0.8771 - val_loss: 1.7215 - val_accuracy: 0.3789
Epoch 5/40
30/30 [==============================] - 158s 5s/step - loss: 0.3344 - accuracy: 0.8625 - val_loss: 0.7209 - val_accuracy: 0.4164
Epoch 6/40
30/30 [==============================] - 167s 6s/step - loss: 0.3291 - accuracy: 0.8672 - val_loss: 1.5914 - val_accuracy: 0.3933
Epoch 7/40
30/30 [==============================] - 157s 5s/step - loss: 0.3312 - accuracy: 0.8704 - val_loss: 1.4666 - val_accuracy: 0.4089
Epoch 8/40
30/